In [3]:
# -*- coding:utf-8 -*-
import pymongo
import time
from bson import ObjectId

def get_mongo_db():
    conn = pymongo.MongoClient('192.168.1.6',27017).phone
    conn.authenticate('phonedata','phonedata1805')
    return conn

# 把本次导出日志插入到 etl_export2result_log 集合
def set_export2result_log(crawl_coll_name, minid, maxid):
    db = get_mongo_db()
    strtime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    doc = {"crawl_coll_name": crawl_coll_name,
          "minid":minid,
          "maxid":maxid,
          "time":strtime}
    db.etl_export2result_log.insert_one(doc)    

# 根据要导出的集合名，查询etl_export2result_log，得到需要增量导出需要的“_id” 起始位置 
# TODO，当前实现只取插入时间最晚的一条记录，后续可以考虑综合多条记录来得到start_id
def get_start_id(crawl_coll_name):
    db = get_mongo_db()
    # maxid = ObjectId('591e9d15c24baefc6a915802')
    maxid = ObjectId('000000000000000000000000')
    
    rslt = db.etl_export2result_log.find({"crawl_coll_name": crawl_coll_name}).sort("_id", pymongo.DESCENDING)
    for rs in rslt:
        maxid = rs["maxid"]
        print('from log, maxid:', maxid)
        break
    print(crawl_coll_name, ' maxid:', maxid)
    return maxid

# 清空所有result collection相关的log 和 结果
def clean_all_exported_result():
    db = get_mongo_db()
    ret = db.drop_collection('result_proidlist')
    print('drop collection status:', ret)
    ret = db.drop_collection('result_comment')
    print('drop collection status:', ret)
    db.etl_export2result_log.delete_many({})

#########################################################################################################
#########################################################################################################
# 导出京东数据到result collection
def export_jd2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',          
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': cur.get('comment_userLeve'),
            'province': '',
            'terminal': '',
            'comment_words': words,
            'comment_score': str(cur.get('comment_score')),
            'comment_date':cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid)
            
    set_export2result_log(crawl_coll_name, minid, maxid) 
    print("etl %s ended." % sitename)

#########################################################################################################
#########################################################################################################
# 导出酷派数据到result collection
def export_cool2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',        
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': '',
            'province': '',
            'terminal': '',
            'comment_words': words,
            'comment_score': cur.get('comment_score'),
            'comment_date': cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid)
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename)    

#########################################################################################################
#########################################################################################################
# 导出国美数据到result collection
def export_guomei2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',       
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': '',
            'province': cur.get('comment_city'),
            'terminal': '',
            'comment_words': words,
            'comment_score': str(cur.get('comment_score')),
            'comment_date': cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid)
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename) 
#########################################################################################################
#########################################################################################################
# 导出华为数据到result collection
def export_huawei2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',        
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': cur.get('comment_userLeve'),
            'province': '',
            'terminal': '',
            'comment_words': words,
            'comment_score': cur.get('comment_score'),
            'comment_date': cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid) 
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename)    
#########################################################################################################
#########################################################################################################
# 导出oppo数据到result collection
def export_oppo2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',         
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': '',
            'province': '',
            'terminal': '',
            'comment_words': words,
            'comment_score': cur.get('comment_score'),
            'comment_date': cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid) 
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename)
#########################################################################################################
#########################################################################################################
# 导出suning数据到result collection
def export_suning2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': cur.get('comment_userLeve'),
            'province': '',
            'terminal': '',
            'comment_words': words,
            'comment_score': str(cur.get('comment_score')),
            'comment_date': cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid) 
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename)
#########################################################################################################
#########################################################################################################
# 导出太平洋数据到result collection
def export_tpy2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',      
            'proid': proid,
            'from': sitename,
            'userid': '',
            'userlevel': '',
            'province': cur.get('user_address'),
            'terminal': '',
            'comment_words': words,
            'comment_score': cur.get('star'),
            'comment_date': cur.get('time'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid) 
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename)    

#########################################################################################################
#########################################################################################################
# 导出小米数据到result collection
def export_xiaomi2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',      
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': '',
            'province': '',
            'terminal': '',
            'comment_words': words,
            'comment_score': '',
            'comment_date': cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid) 
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename)   
#########################################################################################################
#########################################################################################################
# 导出中关村在线数据到result collection
def export_zol2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',        
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': '',
            'province': '',
            'terminal': '',
            'comment_words': words,
            'comment_score': cur.get('comment_score'),
            'comment_date': cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid) 
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename)   
#########################################################################################################
#########################################################################################################
# 导出中兴数据到result collection
def export_zte2result(crawl_coll_name, sitename):
    print('start etl %s...' % sitename)
    db = get_mongo_db()
    coll_crawl_comment = db.get_collection(crawl_coll_name)   
    coll_rstl_proids = db.get_collection('result_proidlist')
    coll_rstl_comment = db.get_collection('result_comment')
    
    minid  = get_start_id(crawl_coll_name)
    maxid = minid
    cursor = coll_crawl_comment.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)    
    proidlist = coll_rstl_proids.distinct('proid',  {'from': sitename})
    for cur in cursor:
        words = cur.get('comment_word')
        proid = cur.get('proid')
        title = cur.get('title')
        if words.strip() == '' or proid.strip() == '': continue
        if title.strip() == '': continue
        doc = {
            'phoneid': '',   
            'proid': proid,
            'from': sitename,
            'userid': cur.get('comment_user'),
            'userlevel': '',
            'province': '',
            'terminal': '',
            'comment_words': words,
            'comment_score': cur.get('comment_score'),
            'comment_date': cur.get('comment_date'),
        }   
        maxid = cur.get('_id')
        coll_rstl_comment.insert_one(doc)
        if proid not in proidlist:
            proid_doc = {
                'phoneid': '',
                'phonename': '',
                'proid': proid,
                'from': sitename,
                'name': title,                
            }
            coll_rstl_proids.insert_one(proid_doc)
            proidlist.append(proid) 
            
    set_export2result_log(crawl_coll_name, minid, maxid)
    print("etl %s ended." % sitename)
    
# result_comment表中新导入的记录增加phoneid字段
# TODO 还未进行严格测试
def append_phoneid2new_record():
    print('append_phoneid2new_record start...')
    db = get_mongo_db()
    coll_proids = db.get_collection('result_proidlist')
    coll_comment = db.get_collection('result_comment')
    proid_docs = coll_proids.find({'phoneid': {'$ne': ''}})    
    for proiddoc in proid_docs:
        print('procing one ...')
        filt = {'proid': proiddoc.get('proid'), 'from': proiddoc.get('from'), 'phoneid': ''}
        toset = {'$set': {'phoneid': proiddoc.get('phoneid')}}
        coll_comment.update_many(filt, toset)
    print('append_phoneid2new_record ended')
    
if __name__ == '__main__':
    export_jd2result('crawl_jd_comment', 'jd')
    export_cool2result('crawl_cool_comment', 'cool')    
    export_guomei2result('crawl_guomei_comment', 'guomei')
    export_huawei2result('crawl_huawei_comment', 'huawei')   
    export_oppo2result('crawl_oppo_comment', 'oppo')
    export_suning2result('crawl_suning_comment', 'suning')   
    export_tpy2result('crawl_tpy_comment', 'tpy')
    export_xiaomi2result('crawl_xiaomi_comment', 'xiaomi')   
    export_zol2result('crawl_zol_comment', 'zol')
    export_zte2result('crawl_zte_comment', 'zte')
    append_phoneid2new_record()


start etl jd...
from log, maxid: 5959a9155c00654fb142d764
crawl_jd_comment  maxid: 5959a9155c00654fb142d764
etl jd ended.
start etl cool...
from log, maxid: 59410512f006764a29a50738
crawl_cool_comment  maxid: 59410512f006764a29a50738
etl cool ended.
start etl guomei...
from log, maxid: 5940a5b6f006764a29a49b56
crawl_guomei_comment  maxid: 5940a5b6f006764a29a49b56
etl guomei ended.
start etl huawei...
from log, maxid: 5940630ef006764a29a475c3
crawl_huawei_comment  maxid: 5940630ef006764a29a475c3
etl huawei ended.
start etl oppo...
from log, maxid: 5940b0e5f006764a29a4b276
crawl_oppo_comment  maxid: 5940b0e5f006764a29a4b276
etl oppo ended.
start etl suning...
from log, maxid: 595611d35c00654fb1429f1e
crawl_suning_comment  maxid: 595611d35c00654fb1429f1e
etl suning ended.
start etl tpy...
from log, maxid: 593b62aae13823ccec03d954
crawl_tpy_comment  maxid: 593b62aae13823ccec03d954
etl tpy ended.
start etl xiaomi...
from log, maxid: 595611c35c00654fb1429e7a
crawl_xiaomi_comment  maxid: 5956

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 

...

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 

procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing one ...
procing 